##### Grupo 2 - ADRIÁN TURIEL CHARRO, HONG XIANG CHEN, PABLO CIDONCHA CÓZAR
# Práctica 1. Resolución de problemas con búsqueda. Parte B.

In [2]:
cd ../Practica1/aima-python

C:\Users\aturi\Ejercicios Python\Practica1\aima-python


In [3]:
from search import *

In [4]:
from itertools import permutations

In [5]:
class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, 
        este coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

In [6]:
class PuzleMates(Problem):
        
    def init(self, initial=None, goal=None):
        # Nums Dados / Operaciones / Resultado
        self.initial = initial
        self.goal = goal

    def actions(self, state):        
        num = state[0]
        relleno = [1,2,3,4,5,6,7,8,9]
        
        mis_permutaciones=list(sorted(permutations(relleno))) # print(len(mis_permutaciones)) imprime: 362880
        
        # Ahora mezclamos esto con num
        accs=list()        
        for w in range(len(mis_permutaciones)):                # para cada combinacion posible
            i=0
            combinaciones = []
            for x in num:                                      # recorro los ceros
                if x == 0:
                    aux = mis_permutaciones[w][i]
                    combinaciones.append(aux)                  # y los cambio
                    i=i+1
                else:
                    combinaciones.append(x)      
            accs.append(combinaciones)                         # guardamos las combinaciones generadas
            
        return accs

    def result(self, state, action):    
        num, op, resultado = state
        
        combinaciones = action
        ret = action[0]                                      
                        
        for i in range(1, len(combinaciones)):
            if op[i-1]=='+':
                ret += combinaciones[i]
            elif op[i-1]=='-':
                ret -= combinaciones[i]
            elif op[i-1]=='*':
                ret *= combinaciones[i]
            elif op[i-1]==':':
                ret /= combinaciones[i]
        
        ret -= resultado
        return num, op, ret 

    def goal_test(self, state):
        #state[2] = ret
        return state[2] == 0
    
    def coste_de_aplicar_accion(self, estado, accion):
        return 1

In [7]:
p = PuzleMates((
    (0,13,0,0,0,12,0,0,11,0,0,0,10),
    ('+','*',':','+','+','*','-','-','+','*',':','-','='),
    66))

In [8]:
p.initial

((0, 13, 0, 0, 0, 12, 0, 0, 11, 0, 0, 0, 10),
 ('+', '*', ':', '+', '+', '*', '-', '-', '+', '*', ':', '-', '='),
 66)

In [9]:
p.result(p.initial, (5,13,8,6,7,12,1,3,11,9,4,2,10))

((0, 13, 0, 0, 0, 12, 0, 0, 11, 0, 0, 0, 10),
 ('+', '*', ':', '+', '+', '*', '-', '-', '+', '*', ':', '-', '='),
 0.0)

In [10]:
p.coste_de_aplicar_accion(p.initial, (5,13,8,6,7,12,1,3,11,9,4,2,10))

1

In [11]:
breadth_first_graph_search(p).solution()

[[1, 13, 2, 4, 7, 12, 5, 8, 11, 3, 6, 9, 10]]

In [14]:
#%%timeit
#breadth_first_tree_search(p).solution()

In [15]:
%%timeit
breadth_first_graph_search(p).solution()

2.92 s ± 169 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
#%%timeit
#depth_first_tree_search(p).solution()

In [ ]:
#%%timeit
#depth_first_graph_search(p).solution()

In [13]:
#%%timeit
#best_first_graph_search(p, linear).solution()

NameError: name 'linear' is not defined

In [ ]:
#%%timeit
#astar_search(p, linear).solution()

In [12]:
#%%timeit
#uniform_cost_search(p).solution()

## Memoria
Hemos ejecutado los distintos algoritmos de búsqueda con el timeit para ver cuál es el más eficiente de todos ellos, y sacar nuestras propias conclusiones.

Para este problema, no se nos ha ocurrido ninguna heurística eficiente, ya que al tratarse de cadenas de operaciones, y "no saber" que cadena de caracteres viene a continuación, si la solución es grande y el número que vamos acumulando es pequeño, siempre vamos a poder multiplicar y sumar, o en caso contrario, si el número es muy grande, puede decrecer mediante divisiones o restas en cualquier momento. Por eso consideramos que proceso para generar la solución consistirá en ir ejecutando todas las combinaciones posibles hasta encontrar una solución, en caso de haberla (se trata de un algoritmo completo).

Basándonos en lo que tenemos solo podemos usar los algoritmos que no prescinden de una heurística, así como: primero en anchura, coste uniforme y primero en profundidad. Aunque solo nos ha llegado a dar una solución para este caso el primero en anchura, para otro podría ser más optimo la búsqueda en profundidad, por ejemplo. Ambos acabarán mirando las mismas combinaciones, solo que en diferentes órdenes. Aunque este algoritmo (anchura) no hace control de repetidos, eso no es problema porque con nuestro código no se genera esas posibilidades. 


## Complejidad
La complejidad del algoritmo de búsqueda en anchura sin repetición es O(r^p) dónde r es el factor de ramificación, y p la profundidad mínima del árbol de búsqueda. Factor de ramificación es: 362880 (Número de combinaciones totales de 1-9) m: 9 (porque es la profundidad mínima hasta encontrar la solución).

O(362880^9) -> (es exponencial?)


In [12]:
%%timeit
breadth_first_graph_search(p).solution()

3.47 s ± 211 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
